# Backups

dbatools is **awesome** at performing backups :-)

The next block sets the variables for tje instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [11]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

Creating connection to the containers

We have a connection to the containers

## Investigate a directory using the SQL Service Account

We can investigate the file system from the viewpoint of the SQL Service Account using `Get-DbaFile` We are going to use that to show the files in the directory but this can be really useful when investigating errors with file system access, for example when backups fail or when reading from shares. This command will try ot access the path from teh SQL Instance using the SQL Account which means that you can troubleshoot the error and also that users do not need access to directories holding backups or client files for example  
If we look in the Directory that we created in the create container notebook using the command below

In [12]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\dummyfile.txt ca2f813564a4

You can see that all there is in the directory is a dummy file that we created earlier when we set up the directories  

## Check the last time that the databases were backed up

Knowing the last time a database was backed up is often useful information and with dbatools it can be retrieved easily. (NB - I use Warning action silently continue to hide the backups from multiple forks warnings that I have created when I created the container and havent fixed yet!)

In [13]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance Database LastFullBackup LastDiffBackup LastLogBacku

p

------------ ------------ ----------- -------- -------------- -------------- ------------

localhost MSSQLSERVER ca2f813564a4 AdventureWorks2017 2019-12-21 14:42:01.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER ca2f813564a4 master 2019-12-21 14:41:56.000

...

localhost MSSQLSERVER 263dec378780 model 2019-12-21 14:41:56.000

localhost MSSQLSERVER 263dec378780 msdb 2019-12-21 14:41:57.000

localhost MSSQLSERVER 263dec378780 Northwind 2019-12-21 14:42:01.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 263dec378780 pubs 2019-12-21 14:42:02.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 263dec378780 WideWorldImporters 2019-12-21 14:42:03.000 2019-12-21 14:42:02.000

The databases were last backed up on the 21st December 2019. Thats not so clever!

## Docker requires permissions

You will get errors for the command below if the account that you have used to allow Docker to access your drives does not have permissions to the folder that we have created. [You can see the instructions here](https://docs.docker.com/docker-for-windows/)

You should add the docker account with full control over your dbatoolsdemo directory in your User Profile directory if you get errors below.

![DockerCompose](.\images\dbatoolsdemopermissions.png )

Its annoying but you can give the docker account modify permissions to your documents folder and it will inherited

![DockerCompose](.\images\documentsdirectorypermissions.png )



## Perform a backup of the entire instance

With one line of code we can quickly backup an entire instance.

In [14]:
Backup-DbaDatabase  -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

ca2f813564a4 model Full 2.59 MB Disk 2020-03-01 15:59:02.000 00:00:00 2020-03-01 15:59:02.000

ca2f813564a4 msdb Full 16.09 MB Disk 2020-03-01 15:59:03.000 00:00:00 2020-03-01 15:59:03.000

ca2f813564a4 Northwind Full 6.71 MB Disk 2020-03-01 15:59:04.000 00:00:00 2020-03-01 15:59:04.000

ca2f813564a4 pubs Full 4.46 MB Disk 2020-03-01 15:59:04.000 00:00:01 2020-03-01 15:59:05.000

ca2f813564a4 WideWorldImporters Full 444.60 MB Disk 2020-03-01 15:59:05.000 00:00:10 2020-03-01 15:59:15.000

All of the databases are backed up - Lets have a look in the directory

In [15]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\AdventureWorks2017_202003011558.bak ca2f813564a4

/var/opt/mssql/backups/SQL1\dummyfile.txt ca2f813564a4

/var/opt/mssql/backups/SQL1\master_202003011559.bak ca2f813564a4

/var/opt/mssql/backups/SQL1\model_202003011559.bak ca2f813564a4

/var/opt/mssql/backups/SQL1\msdb_202003011559.bak ca2f813564a4

/var/opt/mssql/backups/SQL1\Northwind_202003011559.bak ca2f813564a4

/var/opt/mssql/backups/SQL1\pubs_202003011559.bak ca2f813564a4

/var/opt/mssql/backups/SQL1\WideWorldImporters_202003011559.bak ca2f813564a4

If you want to have a look in the mounted volume on your laptop 

In [16]:
Get-ChildItem "$FolderPath\SQL1"

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL1

Mode LastWriteTime Length Name

---- ------------- ------ ----

-a--- 01/03/2020 15:59 217178112 AdventureWorks2017_202003011558.bak

-a--- 01/03/2020 15:56 0 dummyfile.txt

...

-a--- 01/03/2020 15:59 2744320 model_202003011559.bak

-a--- 01/03/2020 15:59 16896000 msdb_202003011559.bak

-a--- 01/03/2020 15:59 7065600 Northwind_202003011559.bak

-a--- 01/03/2020 15:59 4706304 pubs_202003011559.bak

-a--- 01/03/2020 15:59 466231296 WideWorldImporters_202003011559.bak

## Backup to a directory for each database

Rob - I hear you cry, I have 2000 databases, I dont want them all backing up to the same directory - Can dbatools help me there ?

Sure, just add the `-CreateFolder` switch.

Lets demonstrate with SQL2

Theres nothing up my sleeves, lets look at the SQL2 folder


In [17]:
Get-DbaFile -SqlInstance $SQL2 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL2

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL2\dummyfile.txt 263dec378780

Only the dummy file! 

### Backup the databases

In [18]:
Backup-DbaDatabase -SqlInstance $SQL2 -SqlCredential $SqlCredential -CreateFolder -Path /var/opt/mssql/backups/SQL2 -CompressBackup -WarningAction SilentlyContinue

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

263dec378780 model Full 2.59 MB Disk 2020-03-01 15:59:31.000 00:00:00 2020-03-01 15:59:31.000

263dec378780 msdb Full 16.09 MB Disk 2020-03-01 15:59:32.000 00:00:00 2020-03-01 15:59:32.000

263dec378780 Northwind Full 6.71 MB Disk 2020-03-01 15:59:32.000 00:00:01 2020-03-01 15:59:33.000

263dec378780 pubs Full 4.46 MB Disk 2020-03-01 15:59:33.000 00:00:00 2020-03-01 15:59:33.000

263dec378780 WideWorldImporters Full 444.60 MB Disk 2020-03-01 15:59:34.000 00:00:05 2020-03-01 15:59:39.000

## have a look in the directory

In [19]:
Get-ChildItem "$FolderPath\SQL2" -Recurse

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2

Mode LastWriteTime Length Name

---- ------------- ------ ----

d---- 01/03/2020 15:59 AdventureWorks2017

d---- 01/03/2020 15:59 master

...

-a--- 01/03/2020 15:59 520192 pubs_202003011559.bak

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\WideWorldImporters

Mode LastWriteTime Length Name

---- ------------- ------ ----

-a--- 01/03/2020 15:59 126390272 WideWorldImporters_202003011559.bak

As you can see we have the databases backed up, each in their own directory :-)

## Double check all the instances

dbatoos works with multiple instances whereever there is a `-SqlInstance` parameter

Lets check the last time that these databases were backed up on both of the containers

In [20]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance Database LastFullBackup LastDiffBackup LastLogBacku

p

------------ ------------ ----------- -------- -------------- -------------- ------------

localhost MSSQLSERVER ca2f813564a4 AdventureWorks2017 2020-03-01 15:59:00.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER ca2f813564a4 master 2020-03-01 15:59:01.000

...

localhost MSSQLSERVER 263dec378780 model 2020-03-01 15:59:31.000

localhost MSSQLSERVER 263dec378780 msdb 2020-03-01 15:59:32.000

localhost MSSQLSERVER 263dec378780 Northwind 2020-03-01 15:59:33.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 263dec378780 pubs 2020-03-01 15:59:33.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 263dec378780 WideWorldImporters 2020-03-01 15:59:39.000 2019-12-21 14:42:02.000

# We want to use Ola Hallengren for our backups

Thats ok, dbatools supports many of the excellent community tooling including Ola Hallengrens solution.

You can install Ola Hallengren like this (except in containers)

````
$installDbaMaintenanceSolutionSplat = @{
    SqlInstance = $sql1
    InstallJobs = $true
    ReplaceExisting = $true
    LogToTable = $true
    OutputFileDirectory = '/var/opt/mssql/backups/SQL1'
    BackupLocation = '/var/opt/mssql/backups/SQL1'
    CleanupTime = 72
    Solution = 'All'
    Database = 'master'
}
Install-DbaMaintenanceSolution @installDbaMaintenanceSolutionSplat -Verbose
````

Lets have a look at the Agent Jobs on the Instance

In [21]:
Get-DbaAgentJob -SqlInstance $sql1 |Select Name

Name

----

CommandLog Cleanup

DatabaseBackup - SYSTEM_DATABASES - FULL

DatabaseBackup - USER_DATABASES - DIFF

...

DatabaseIntegrityCheck - USER_DATABASES

IndexOptimize - USER_DATABASES

sp_delete_backuphistory

sp_purge_jobhistory

The Beard is Important

Those look like Ola Jobs to me :-)

Lets run the jobs to back up the system and user databases

In [23]:
$Jobs = 'DatabaseBackup - SYSTEM_DATABASES - FULL' ,'DatabaseBackup - USER_DATABASES - FULL'
Start-DbaAgentJob -SqlInstance $sql1 -Job $Jobs

WARNING: [16:17:48][Start-DbaAgentJob] DatabaseBackup - SYSTEM_DATABASES - FULL on [localhost,15592] is not idle (Executing)

WARNING: [16:17:48][Start-DbaAgentJob] DatabaseBackup - USER_DATABASES - FULL on [localhost,15592] is not idle (Executing)

Check the status of the jobs

In [26]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaAgentJob -SqlInstance $sql1 -Job $Jobs | Select SqlInstance, Name, CurrentRunStatus, LastRunOutCome

SqlInstance Name CurrentRunStatus LastRunOutcome

----------- ---- ---------------- --------------

ca2f813564a4 DatabaseBackup - SYSTEM_DATABASES - FULL Idle Succeeded

ca2f813564a4 DatabaseBackup - USER_DATABASES - FULL Idle Succeeded

When the jobs have finished, lets have a look at the files.
We need to alter the file path to include the current container name from the results above

In [28]:
Get-DbaFile -SqlInstance $sql1 -Path '/var/opt/mssql/data/ca2f813564a4/AdventureWorks2017/FULL'

Filename SqlI

nsta

nce

-------- ----

/var/opt/mssql/data/ca2f813564a4/AdventureWorks2017/FULL\ca2f813564a4_AdventureWorks2017_FULL_20200301_161036.bak ca2…

/var/opt/mssql/data/ca2f813564a4/AdventureWorks2017/FULL\ca2f813564a4_AdventureWorks2017_FULL_20200301_161743.bak ca2…

# Restores

Backups are all very well and good but what about restores?

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason

In [41]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : ca2f813564a4

Database : WideWorldImporters

Status : Dropped

...

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : ca2f813564a4

Database : AdventureWorks2017

Status : Dropped

In [42]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

All the databases have gone - How easy it is to restore them?

This easy :-)

One line of code

## First lets restore from the backups we took with dbatools in, with all files in the one directory

In [43]:
Write-Output "Starting Restoring"
Restore-DbaDatabase  -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 
Write-Output "Finished"

In [45]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

Name

----

WideWorldImporters

pubs

Northwind

AdventureWorks2017

That was easy - WHat about if I have used Ola Hallengren?

AGAIN

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [46]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : ca2f813564a4

Database : WideWorldImporters

Status : Dropped

...

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : ca2f813564a4

Database : AdventureWorks2017

Status : Dropped

## Restore from Ola Halengren Backups 

- Use the path from the above files check with the new container name 

In [47]:
Write-Output "Starting Restoring"
Restore-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/data/ca2f813564a4 -MaintenanceSolutionBackup
Write-Output "Finished"

Starting Restoring

In [48]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

Name

----

WideWorldImporters

Northwind

pubs

AdventureWorks2017

Thats all very well and good

## When were the databases restored?

and which backup was used?

In [49]:
Get-DbaDbRestoreHistory -SqlInstance $SQL1 -Last

BackupFinishDate : 01/03/2020 16:17:44

ComputerName : localhost

Database : AdventureWorks2017

Date : 01/03/2020 16:33:21

From : /var/opt/mssql/data/ca2f813564a4/AdventureWorks2017/FULL/ca2f813564a4_AdventureWorks2017_FULL_20200301_161743.bak

...

InstanceName : MSSQLSERVER

RestoreType : Database

SqlInstance : ca2f813564a4

To : /var/opt/mssql/data/WideWorldImporters.mdf, /var/opt/mssql/data/WideWorldImporters_UserData.ndf, /var/opt/mssql/data/WideWorldImporters.ldf

Username : sqladmin

# Don't forget to clean up

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up